In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
import time

In [ ]:
df_oil = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
df_holidays = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
df_train = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv",index_col='id')
df_stores = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
df_transactions = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")
df_test = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv",index_col='id')
df_sample = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv")

In [ ]:
print('Train: ',min(df_train.date),max(df_train.date))
print('Test: ',min(df_test.date),max(df_test.date))

In [ ]:
def summary(df):
    print(f'data shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['#missing'] = df.isnull().sum().values 
    summ['%missing'] = df.isnull().sum().values / len(df)* 100
    summ['#unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    summ['first value'] = df.loc[0].values
    summ['second value'] = df.loc[1].values
    summ['third value'] = df.loc[2].values
    
    return summ

In [ ]:
df_oil.head()

In [ ]:
summary(df_oil)

In [ ]:
# Выводим строки с пропущенными значениями до интерполяции
print("Before interpolation:")
print(df_oil[df_oil['dcoilwtico'].isna()])

# Интерполируем пропущенные значения
df_oil['dcoilwtico'] = df_oil['dcoilwtico'].interpolate()
# Этот код использует метод interpolate для заполнения пропущенных числовых значений
# между ближайшими не-NA значениями с использованием линейной интерполяции.

# Выводим строки с пропущенными значениями после интерполяции
print("\nAfter interpolation:")
print(df_oil[df_oil['dcoilwtico'].isna()])

In [ ]:
df_oil['date'] = pd.to_datetime(df_oil['date'])

In [ ]:
df_oil.loc[df_oil['dcoilwtico'].isna(), 'dcoilwtico'] = 93.14

In [ ]:
summary(df_oil)

In [ ]:
df_stores.head(2)

In [ ]:
summary(df_stores)

In [ ]:
df_train.head(2)

In [ ]:
summary(df_train)

In [ ]:
df_train['date'] = pd.to_datetime(df_train['date'])

In [ ]:
df_test['date'] = pd.to_datetime(df_test['date'])

In [ ]:
df_train_start = df_train.date.min().date()
df_train_end = df_train.date.max().date()

missing_dates = pd.date_range(df_train_start, df_train_end
                             ).difference(df_train.date.unique())
missing_dates = missing_dates.strftime("%Y-%m-%d").tolist()

missing_dates

In [ ]:
df_test_start = df_test.date.min().date()
df_test_end = df_test.date.max().date()

missing_dates_test = pd.date_range(df_test_start, df_test_end
                             ).difference(df_test.date.unique())
missing_dates_test = missing_dates_test.strftime("%Y-%m-%d").tolist()

missing_dates_test

In [ ]:
# reindex training data
multi_idx = pd.MultiIndex.from_product([pd.date_range(df_train_start, df_train_end), 
                                        df_train.store_nbr.unique(), 
                                        df_train.family.unique()],
                                        names=["date", "store_nbr", "family"],)
df_train = df_train.set_index(["date", "store_nbr", "family"]
                                 ).reindex(multi_idx).reset_index()
# fill missing values with 0s
df_train[["sales", "onpromotion"]] = df_train[["sales", "onpromotion"]].fillna(0.)

In [ ]:
df_transactions.head()

In [ ]:
df_transactions['date'] = pd.to_datetime(df_transactions['date'])

In [ ]:
summary(df_transactions)

In [ ]:
num_store = df_train.store_nbr.nunique()
train_len = (df_train_end - df_train_start).days + 1

num_zero_sales = (df_train.groupby(["date", "store_nbr"]).sales.sum().eq(0)).sum()
total_rec = num_store * train_len
curr_rec = len(df_transactions.index)
missing_rec = total_rec - curr_rec - num_zero_sales

#total sales for each store
store_sales = df_train.groupby(["date", "store_nbr"]).sales.sum().reset_index()

# reindex transaction data
df_transactions = df_transactions.merge(
    store_sales, on=["date", "store_nbr"],how="outer").sort_values(
    ["date", "store_nbr"],ignore_index=True)

# fill missing values with 0s for days with zero sales
df_transactions.loc[df_transactions.sales.eq(0), "transactions"] = 0
df_transactions = df_transactions.drop(columns=["sales"])

# fill remaining missing values using linear interpolation
df_transactions.transactions = df_transactions.groupby(
    "store_nbr", group_keys=False).transactions.apply(
    lambda x: x.interpolate(method="linear", limit_direction="both"))

In [ ]:
df_holidays.head(2)

In [ ]:
df_holidays['date'] = pd.to_datetime(df_holidays['date'])

In [ ]:
print('Holidays: ',min(df_holidays.date),max(df_holidays.date))

In [ ]:
df_holidays['type'].unique()

In [ ]:
df_holidays['description'].unique()

In [ ]:
df_oil = df_oil.merge(pd.DataFrame({"date": pd.date_range(df_train_start, 
                                                              df_test_end)}),
                          on="date",how="outer",).sort_values("date", ignore_index=True)

In [ ]:
summary(df_oil)

In [ ]:
df_oil.dcoilwtico = df_oil.dcoilwtico.interpolate(method="linear", limit_direction="both")

In [ ]:
def process_holiday(s):
    if "futbol" in s:
        return "futbol"
    to_remove = list(set(df_stores['city'].str.lower()) | set(df_stores['state'].str.lower()))
    for w in to_remove:
        s = s.replace(w, "")
    return s

df_holidays['description'] = df_holidays.apply(
    lambda x: x['description'].lower().replace(x['locale_name'].lower(), ""), 
    axis=1,).apply(process_holiday).replace(
    r"[+-]\d+|\b(de|del|traslado|recupero|puente|-)\b", "", regex=True,).replace(
    r"\s+|-", " ", regex=True,).str.strip()

# remove transferred holidays
df_holidays = df_holidays[df_holidays['transferred'].eq(False)]

#Saturdays designated as work days 
work_days = df_holidays[df_holidays['type'].eq("Work Day")]
work_days = work_days[["date", "type"]].rename(columns={"type": "work_day"}
                                              ).reset_index(drop=True)
work_days['work_day'] = work_days['work_day'].notna().astype(int)

# remove work days after extracting above
df_holidays = df_holidays[df_holidays['type']!="Work Day"].reset_index(drop=True)
df_holidays

In [ ]:
#local holidays (city level) 
local_holidays = df_holidays[df_holidays['locale'].eq("Local")]
local_holidays = local_holidays[["date", "locale_name", "description"]].rename(
    columns={"locale_name": "city"}).reset_index(drop=True)

local_holidays = local_holidays[~local_holidays.duplicated()]
local_holidays = pd.get_dummies(local_holidays, columns=["description"], prefix="loc")

local_holidays.head()

In [ ]:
#regional holidays 
regional_holidays = df_holidays[df_holidays['locale'].eq("Regional")]
regional_holidays = regional_holidays[["date", "locale_name", "description"]].rename(
    columns={"locale_name": "state", "description": "provincializacion"}).reset_index(drop=True)
regional_holidays['provincializacion'] = regional_holidays['provincializacion'].eq(
    "provincializacion").astype(int)

regional_holidays

In [ ]:
#national holidays 
national_holidays = df_holidays[df_holidays['locale'].eq("National")]
national_holidays = national_holidays[["date", "description"]].reset_index(drop=True)
national_holidays = national_holidays[~national_holidays.duplicated()]
national_holidays = pd.get_dummies(national_holidays, columns=["description"], prefix="nat")

# different national holidays may fall on the same day
national_holidays = national_holidays.groupby("date").sum().reset_index()
# shorten name for visualization purposes later
national_holidays = national_holidays.rename(columns={
    "nat_primer grito independencia": "nat_primer grito"})

national_holidays.head()

In [ ]:
national_holidays.columns

In [ ]:
sales_ts = pd.pivot_table(df_train, values="sales", index="date", 
                          columns=["store_nbr", "family"])
tr_ts = pd.pivot_table(df_transactions, values="transactions", index="date", 
                       columns="store_nbr")
promo_ts = pd.pivot_table(df_train, values="onpromotion", index="date", 
                          columns=["store_nbr", "family"])

In [ ]:
%%time
from sklearn.preprocessing import MinMaxScaler
# scale target series
scaler = MinMaxScaler()
sales_ts_scaled = sales_ts.copy()
sales_ts_scaled[sales_ts_scaled.columns] = scaler.fit_transform(sales_ts_scaled)

# convert back to long form and add the holiday columns
holiday_sales_merged = sales_ts_scaled.melt(
    value_name="sales", ignore_index=False,).reset_index().merge(
    df_stores, on="store_nbr", how="left").merge(
    work_days, on="date", how="left").merge(
    local_holidays, on=["date", "city"], how="left").merge(
    regional_holidays, on=["date", "state"], how="left").merge(
    national_holidays, on="date", how="left").fillna(0)

# include dummy variable for dates without any holidays
holiday_list = [col for col in holiday_sales_merged if col.startswith((
    "loc_", "nat_", "provincializacion"))]
holiday_sales_merged["no_holiday"] = holiday_sales_merged[holiday_list].sum(
    axis=1).eq(0).astype(int)

holiday_sales_merged.head()

In [ ]:
%%time
# keep selected national holidays with larger impacts on sales
selected_holidays = ['nat_batalla', 'nat_black friday', 'nat_carnaval',
       'nat_cyber monday', 'nat_dia difuntos', 'nat_dia la madre',
       'nat_dia trabajo', 'nat_futbol', 'nat_independencia', 'nat_navidad',
       'nat_primer dia ano', 'nat_primer grito', 'nat_terremoto',
       'nat_viernes santo']
keep_national_holidays = national_holidays[["date", *selected_holidays]]

data = pd.concat(
    [df_train, df_test], axis=0, ignore_index=True,
).merge(
    df_stores, on=["store_nbr"]
).merge(
    df_oil, on=["date"], how="left"
).merge(
    df_transactions, on=["date", 'store_nbr'], how="left"
).merge(
    work_days, on="date", how="left",    
).merge(
    keep_national_holidays, on=["date"],how="left").sort_values(["date", "store_nbr", "family"], ignore_index=True)

data[["work_day", *selected_holidays]] = data[["work_day", *selected_holidays]].fillna(0)

In [ ]:
## Select the date, days of the week, hours, month  !not used in calculations
data['day_of_week'] = data.date.dt.dayofweek
data['day_of_year'] = data.date.dt.dayofyear
data['day_of_month'] = data.date.dt.day
data['year'] = data.date.dt.year
data['month'] = data.date.dt.month
#seasons 0-winter;1-spring;2-summer;3-fall
data["season"] = np.where(data.date.dt.month.isin([12,1,2]), 0, 1)
data["season"] = np.where(data.date.dt.month.isin([3,4,5]), 1, data["season"])
data["season"] = np.where(data.date.dt.month.isin([6,7,8]), 2, data["season"])
data["season"] = np.where(data.date.dt.month.isin([9, 10, 11]), 3, data["season"])
data.head()

In [ ]:
data['type'].unique()

In [ ]:
data['type'] = data['type'].replace(['A', 'B', 'C', 'D', 'E'], ['1', '2', '3', '4', '5'])
data['type'] = data['type'].astype(int)

In [ ]:
data.columns

16 апреля 2016 года в Эквадоре произошло землетрясение магнитудой 7,8. Люди объединились для оказания помощи, жертвуя воду и другие товары первой необходимости, что сильно повлияло на продажи в супермаркетах в течение нескольких недель после землетрясения. Это необходимо учесть.

In [ ]:
condition = (data['date'] < '2016-04-16') | (data['date'] > '2016-05-15')
data = data.loc[condition]

In [ ]:
# Фильтрация данных для начала каждого года
sales_start_of_year = data[data['date'].dt.month.isin([1, 2]) & (data['date'].dt.day == 1)]

# Группировка данных по магазинам и подсчет продаж
sales_by_store = sales_start_of_year.groupby(['store_nbr', train_df['date'].dt.year])['sales'].sum().reset_index()

# Отображение уникальных значений store_nbr, где продажи равны 0 в течение нескольких месяцев каждого года
stores_with_zero_sales = sales_by_store[sales_by_store['sales'] == 0]['store_nbr'].unique()

# Вывод результатов
print("Магазины с продажами 0 в течение нескольких месяцев каждого года:", stores_with_zero_sales)

In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go

# Фильтрация данных для выбранных магазинов
selected_stores = [20, 21, 22, 29, 36, 42, 52, 53]
filtered_data = data[data['store_nbr'].isin(selected_stores)]

# Создание сетки графиков
fig = sp.make_subplots(rows=len(selected_stores), cols=1, subplot_titles=[f"Магазин {store}" for store in selected_stores],
                       shared_xaxes=True, vertical_spacing=0.05)

# Добавление линейных графиков для каждого магазина
for i, store in enumerate(selected_stores, start=1):
    store_data = filtered_data[filtered_data['store_nbr'] == store]
    trace = go.Scatter(x=store_data['date'], y=store_data['sales'], mode='lines', name=f'Магазин {store}')
    fig.add_trace(trace, row=i, col=1)

# Настройка макета и отображение графиков
fig.update_layout(title_text='Продажи в выбранных магазинах', showlegend=False)
fig.update_xaxes(title_text='Дата', row=len(selected_stores), col=1)
fig.update_yaxes(title_text='Продажи', col=1)
fig.show()

In [ ]:
print(data.shape)
data = data[~((data.store_nbr == 20) & (data.date < "2015-02-13"))]
data = data[~((data.store_nbr == 21) & (data.date < "2015-07-24"))]
data = data[~((data.store_nbr == 22) & (data.date < "2015-10-09"))]
data = data[~((data.store_nbr == 29) & (data.date < "2015-03-20"))]
data = data[~((data.store_nbr == 36) & (data.date < "2013-05-09"))]
data = data[~((data.store_nbr == 42) & (data.date < "2015-08-21"))]
data = data[~((data.store_nbr == 52) & (data.date < "2017-04-20"))]
data = data[~((data.store_nbr == 53) & (data.date < "2014-05-29"))]
print(data.shape)

In [ ]:
# Calculate the total sales for each store
store_sales = data.groupby('store_nbr')['sales'].sum().reset_index()

# Sort the stores based on sales in descending order
store_sales = store_sales.sort_values('sales', ascending=False)

# Create a bar plot
plt.figure(figsize=(12, 6))
sns.barplot(data=store_sales, x='store_nbr', y='sales')

plt.xlabel('Store Number')
plt.ylabel('Total Sales')
plt.title('Total Sales by Store')

plt.xticks(rotation=45)

plt.show()

In [ ]:
import itertools

# Group the data by month, year, and calculate the total sales
monthly_sales = data.groupby(['month', 'year'])['sales'].sum().reset_index()

# Create the line chart
plt.figure(figsize=(10, 6))  # Set the figure size

# Get unique years and cycle through colors
years = monthly_sales['year'].unique()
colors = itertools.cycle(['red', 'green', 'blue', 'orange', 'purple'])

for year in years:
    year_data = monthly_sales[monthly_sales['year'] == year]
    plt.plot(year_data['month'], year_data['sales'], marker='o', color=next(colors), label=str(year))

plt.xlabel('Month')
plt.ylabel('Sales')
plt.title('Monthly Sales Trend')

# Customize x-axis ticks to show month names
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
plt.xticks(range(1, 13), month_names)


plt.legend()

plt.show()

In [ ]:
sales_series = data['sales']
autocorr_values = sales_series.autocorr()
print("Autocorrelation:", autocorr_values)

In [ ]:
data.columns

In [ ]:
data.sample(5)

In [ ]:
FEATURES = ['store_nbr', 'onpromotion', 'type', 'cluster',
       'dcoilwtico', 'transactions', 'work_day', 'nat_batalla',
       'nat_black friday', 'nat_carnaval', 'nat_cyber monday',
       'nat_dia difuntos', 'nat_dia la madre', 'nat_dia trabajo', 'nat_futbol',
       'nat_independencia', 'nat_navidad', 'nat_primer dia ano',
       'nat_primer grito', 'nat_terremoto', 'nat_viernes santo', 'day_of_week',
       'day_of_year', 'day_of_month', 'year', 'month', 'season']

no_features = ('date', 'family', 'sales', 'city','state')

for i in no_features:
    FEATURES.remove(i)

TARGET = 'sales'

In [ ]:
train = data[data['sales'].notnull()].copy()
test = data[data['sales'].isnull()].drop(['sales'],axis=1)

In [ ]:
params = {'learning_rate': 0.01,
          'n_estimators': 1000, 
          'max_depth': 15,           
          'lambda': 6, 
          'alpha': 1, 
          'colsample_bytree': 0.5, 
          'subsample': 0.7, 
          'min_child_weight': 155,
          'early_stopping_rounds': 10}

In [ ]:
%%time

from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error

cat_predictions = np.array([])
xgb_predictions = np.array([])

y_val_cat = pd.DataFrame()
y_val_xgb = pd.DataFrame()

cat = CatBoostRegressor()
xgb = XGBRegressor(**params)

test_predict = pd.DataFrame()

cat_submit = np.array([])
xgb_submit = np.array([])

# Группировка по столбцу 'family'
for family, family_data in train.groupby('family'):
    X_train_family, X_val_family, y_train_family, y_val_family = train_test_split(
        family_data, family_data[TARGET], test_size=0.05, shuffle=False)

    cat.fit(X_train_family[FEATURES], y_train_family, eval_set=[(X_train_family[FEATURES], y_train_family),
                                                               (X_val_family[FEATURES], y_val_family)],
            verbose=False, early_stopping_rounds=10)

    xgb.fit(X_train_family[FEATURES], y_train_family, eval_set=[(X_train_family[FEATURES], y_train_family),
                                                                (X_val_family[FEATURES], y_val_family)],
            verbose=False)

    cat_pred_family = cat.predict(X_val_family[FEATURES])
    cat_pred_family = np.maximum(cat_pred_family, 0)  # Заменяем отрицательные значения на 0
    cat_predictions = np.concatenate((cat_predictions, cat_pred_family))
    y_val_cat = pd.concat([y_val_cat, y_val_family])

    xgb_pred_family = xgb.predict(X_val_family[FEATURES])
    xgb_pred_family = np.maximum(xgb_pred_family, 0)  # Заменяем отрицательные значения на 0
    xgb_predictions = np.concatenate((xgb_predictions, xgb_pred_family))
    y_val_xgb = pd.concat([y_val_xgb, y_val_family])

    test_family = test.loc[test['family'] == family]

    cat_pred_submit = cat.predict(test_family[FEATURES])
    cat_pred_submit = np.maximum(cat_pred_submit, 0)  # Заменяем отрицательные значения на 0
    cat_submit = np.concatenate((cat_submit, cat_pred_submit))

    xgb_pred_submit = xgb.predict(test_family[FEATURES])
    xgb_pred_submit = np.maximum(xgb_pred_submit, 0)  # Заменяем отрицательные значения на 0
    xgb_submit = np.concatenate((xgb_submit, xgb_pred_submit))

    test_predict = pd.concat([test_predict, test_family])

    print(family, 'CatBoost RMSLE:', np.sqrt(mean_squared_log_error(y_val_family, cat_pred_family)))
    print(family, 'XGB RMSLE:', np.sqrt(mean_squared_log_error(y_val_family, xgb_pred_family)))


In [ ]:
y_val_cat['pred'] = cat_predictions
y_val_xgb['pred'] = xgb_predictions

In [ ]:
y_val_cat

In [ ]:
y_val_xgb

In [ ]:
print('Overall CatBoost RMSLE:', np.sqrt(mean_squared_log_error(y_val_cat['sales'], y_val_cat['pred'])))
print('Overall XGB RMSLE:', np.sqrt(mean_squared_log_error(y_val_xgb['sales'], y_val_xgb['pred'])))

In [ ]:
test.head(2)

In [ ]:
predictions = [0.5 * a + 0.5 * b for a,b in zip(xgb_submit,cat_submit)] 

In [ ]:
test_predict['pred'] = predictions
test_predict.sort_index(inplace=True)

In [ ]:
test_predict

In [ ]:
df_sample = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv',index_col='id')
df_sample['sales']= np.expm1(test_predict['pred'])

In [ ]:
df_sample

In [ ]:
# df_sample.to_csv('submission.csv')